In [44]:
#### TestPython_31-03_2 --- SentimentAnalysis_CreatingNeuralNetwork ####
# Changing the model for the sentiment analysis pickle and tweaking initial NN

In [45]:
# input> weight> to hidden layer 1 (act. fn)> +weights > hidden layer 2(act. fn)> repeats 
# Feed forward NN
# Compare the o/p with intended o/p > cost fn.(cross entropy)
# Optimization fn. (Optimizer) ->Minimize the cost (Adam Optimizer...SGD...Adagradboost...)
# This will be done by backpropogation
# Feed forward + backprop ---> "One epoch"
# Repeat epochs till it reduces the cost.. and then stabilizes

In [35]:
# from tensorflow.examples.tutorials.mnist import input_data
# creating the same python file for the pickle file
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter
lemmatizer=WordNetLemmatizer()
num_lines=10000000
def create_Lexicon(pos,neg):
    lexicon=[]
    for fi in [pos,neg]:
        with open(fi,'r') as f:
            contents=f.readlines()
            for l in contents[:num_lines]:
                all_words=word_tokenize(l.lower())
                lexicon+=list(all_words)
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    # w_counts = {'the': 12345, 'and': 54321}
    l2 = []
    # l2 is the list of the not super-common but > 50 occurence words
    for w in w_counts:
        # removing only the words which are not super-common and using only the okayish frequent words.
        if 1000 > w_counts[w] > 10:
            l2.append(w)
    print(len(l2))
    return l2
def sample_handling(sample,lexicon,classification):
    featureset=[]
    with open(sample,'r') as f:
        contents = f.readlines()
        for l in contents[:num_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1
            features = list(features)
            featureset.append([features,classification])
    return featureset
def create_featureset_and_labels(pos,neg,test_size=0.1):
    lexicon = create_Lexicon(pos,neg)
    features=[]
    features += sample_handling('SentimentAnalysisData/pos.txt',lexicon,[1,0])
    features += sample_handling('SentimentAnalysisData/neg.txt',lexicon,[0,1])
    random.shuffle(features)
    features=np.array(features)
    testing_size = int(test_size*len(features))
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])
    
    return train_x,train_y,test_x,test_y
if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_featureset_and_labels('SentimentAnalysisData/pos.txt','SentimentAnalysisData/neg.txt')
    with open('sentiment_set_nn.pickle','wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y],f)

2020


In [36]:
# mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)

In [37]:
# This is a multiclass classification of 10 classes... (0-9) class...
# Need for one-hot (0=[1,0,0,0,0,0,0,0,0,0],1=[0,1,0,0,0,0,0,0,0,0] ...)

In [38]:
n_nodes_hl1=500
n_nodes_hl2=500
n_nodes_hl3=500
n_classes=2
batch_size=100

In [39]:
# Each matrix will be height * width

In [40]:
x=tf.placeholder('float',[None,len(train_x[0])])
y=tf.placeholder('float')

In [41]:
# Defining the NN :: (Input data * Weights) + Bias
def nn_model(data):
    # defining the hidden layers and the output layer! (Weights ad biases)
    hl_1={
        'weights':tf.Variable(tf.random_normal([len(train_x[0]),n_nodes_hl1])),
        'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))
    }
    hl_2={
        'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
        'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))
    }
    hl_3={
        'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
        'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))
    }
    opl={
        'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
        'biases':tf.Variable(tf.random_normal([n_classes]))
    }
    l1=tf.add(tf.matmul(data,hl_1['weights']),hl_1['biases'])
    # relu is the rectified linear value (Activation function)
    l1=tf.nn.relu(l1)
    #send the data of layer 1 to layer 2
    l2=tf.add(tf.matmul(l1,hl_2['weights']),hl_2['biases'])
    l2=tf.nn.relu(l2)
    l3=tf.add(tf.matmul(l2,hl_3['weights']),hl_3['biases'])
    l3=tf.nn.relu(l3)
    # output layer: not similar to l1,l2,l3
    output=tf.matmul(l3,opl['weights'])+opl['biases']
    return output

In [42]:
# running data through the nn_model
def train_nn(x):
    prediction = nn_model(x)
    # finding the cost of the nn model
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    # we want to minimize the cost of the final NN
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    #setting num of epochs (FF + BP total)
    num_epochs=19
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(num_epochs):
            epoch_loss=0
            i=0
            while i<len(train_x):
                start = i
                end = i+batch_size
                batch_x=np.array(train_x[start:end])
                batch_y=np.array(train_y[start:end])                
                _ , c = sess.run([optimizer,cost],feed_dict={x: batch_x , y: batch_y})
                epoch_loss += c
                i += batch_size
            print('\nEpoch:',epoch+1,'completed out of',num_epochs,'\nEpoch Loss:',epoch_loss)
        # checking the accuracy on the test images!
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

In [43]:
train_nn(x)


Epoch: 1 completed out of 19 
Epoch Loss: 308218.0456542969

Epoch: 2 completed out of 19 
Epoch Loss: 137315.4803466797

Epoch: 3 completed out of 19 
Epoch Loss: 73102.39758300781

Epoch: 4 completed out of 19 
Epoch Loss: 39709.16719818115

Epoch: 5 completed out of 19 
Epoch Loss: 26041.821071624756

Epoch: 6 completed out of 19 
Epoch Loss: 33084.365958213806

Epoch: 7 completed out of 19 
Epoch Loss: 21750.51486968994

Epoch: 8 completed out of 19 
Epoch Loss: 16204.545599102974

Epoch: 9 completed out of 19 
Epoch Loss: 8954.821391642094

Epoch: 10 completed out of 19 
Epoch Loss: 2922.4926240587233

Epoch: 11 completed out of 19 
Epoch Loss: 1922.2091245651245

Epoch: 12 completed out of 19 
Epoch Loss: 677.484820146587

Epoch: 13 completed out of 19 
Epoch Loss: 235.634618371209

Epoch: 14 completed out of 19 
Epoch Loss: 162.71627555654

Epoch: 15 completed out of 19 
Epoch Loss: 124.87662774303541

Epoch: 16 completed out of 19 
Epoch Loss: 92.99425407612244

Epoch: 17 comp